In [1]:
import os
from dontpatronizeme.semeval_2022 import dont_patronize_me as dpm

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
Hub version:  0.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


2021-12-10 14:23:58.851742: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.


Metal device set to: Apple M1 Max


2021-12-10 14:23:58.851855: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# - <par_id> is a unique id for each one of the paragraphs in the corpus.
# - <art_id> is the document id in the original NOW corpus (News on Web: https://www.english-corpora.org/now/).
# - <keyword> is the search term used to retrieve texts about a target community.
# - <country_code> is a two-letter ISO Alpha-2 country code for the source media outlet.
# - <text> is the paragraph containing the keyword.
# - <label> is an integer between 0 and 4. Each paragraph has been annotated by two annotators as 0 (No PCL), 1 (borderline PCL) and 2 (contains PCL). The combined annotations have been used in the following graded scale:

data = pd.read_csv('dontpatronizeme/dontpatronizeme_v1.4/dontpatronizeme_pcl.tsv', 
                   skiprows=4,
                   sep='\t', header=None)

data.columns = ['par_id', 'art_id', 'keyword', 'country_code', 'text', 'label']
print(data.shape)
data.head(5)

(10469, 6)


,par_id,art_id,keyword,country_code,text,label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0
2,3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0
4,5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0


In [3]:
par_id = data.par_id.to_numpy()
par_id

array([    1,     2,     3, ..., 10467, 10468, 10469])

In [4]:
art_id = data.art_id.to_numpy()
art_id

array(['@@24942188', '@@21968160', '@@16584954', ..., '@@20282330',
       '@@16753236', '@@16779383'], dtype=object)

In [5]:
keyword = data.keyword.to_numpy()
keyword

array(['hopeless', 'migrant', 'immigrant', ..., 'in-need', 'hopeless',
       'homeless'], dtype=object)

In [6]:
country_code = data.country_code.to_numpy()
country_code

array(['ph', 'gh', 'ie', ..., 'ng', 'in', 'ie'], dtype=object)

In [7]:
text = data.text.to_numpy()
text[:2]

array(["We 're living in times of absolute insanity , as I 'm pretty sure most people are aware . For a while , waking up every day to check the news seemed to carry with it the same feeling of panic and dread that action heroes probably face when they 're trying to decide whether to cut the blue or green wire on a ticking bomb -- except the bomb 's instructions long ago burned in a fire and imminent catastrophe seems the likeliest outcome . It 's hard to stay that on-edge for that long , though , so it 's natural for people to become inured to this constant chaos , to slump into a malaise of hopelessness and pessimism .",
       "In Libya today , there are countless number of Ghanaian and Nigerian immigrants . These are the two countries with key macroeconomic challenges including unemployment . Let 's tackle this issue from the root and not the fruit . Thank you"],
      dtype=object)

In [8]:
label = data.label.to_numpy()
label

array([0, 0, 0, ..., 3, 4, 3])

In [9]:
print(np.unique(label, return_counts=True))
# print(np.info(label))

(array([0, 1, 2, 3, 4]), array([8529,  947,  144,  458,  391]))


In [10]:
x_data = list(data.iloc[:, 4].to_numpy())
# x_data

In [11]:
y_label = data.iloc[:, -1].to_numpy()
y_label

array([0, 0, 0, ..., 3, 4, 3])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.20, random_state=42)
# X_train

In [ ]:
cls = dpm.DontPatronizeMe()

In [13]:
X_train = np.array(X_train)
X_test = np.array(X_test)
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(X_train[:3])


2021-12-10 14:23:58.951033: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-10 14:23:58.951055: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-12-10 14:23:58.970673: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-10 14:23:58.979085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-10 14:23:58.989402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.5178323 ,  0.2579619 ,  1.8962922 , -0.22883563,  0.41169694,
        -2.8806913 ,  0.76954746,  1.1166486 , -0.7982834 , -0.71134603,
         1.3698522 ,  0.6062772 , -1.6364819 ,  0.52668756, -1.756137  ,
         1.7023653 ,  2.1843994 , -0.91079664, -0.9670054 , -0.9411651 ],
       [ 2.1064057 ,  0.62601954,  1.4203571 , -1.2698228 ,  1.4543746 ,
        -1.8965639 ,  1.21231   , -0.19299242, -1.1244173 , -0.6563414 ,
         0.50978845,  0.43793494, -2.7593758 ,  0.25953847, -2.0184455 ,
        -0.21656777,  2.050317  , -0.81915814, -0.17326725, -0.01286737],
       [-0.2304229 , -1.6790617 ,  0.2406589 , -1.1624727 , -1.9014754 ,
         0.507259  , -0.9007998 , -0.02438426,  1.2534747 , -0.21496792,
        -1.1290158 , -0.01982248,  0.53049296, -0.4629475 , -0.4677253 ,
         0.39006478,  0.70096076,  0.4863963 , -0.41430557, -0.19327596]],
      dtype=float32)>

In [14]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
# print(X_train.shape)
# print(X_test.shape)
# print(X_train[:2])
# print(y_train, y_train.shape)
# print(y_test, y_test.shape)

# for ii, x in enumerate(X_train):
#     X_train[ii] = x.encode()
# for ii, x in enumerate(X_test):
#     print(X_test[ii])
#     X_test[ii] = x.encode()



In [17]:
%%time
# ~1 minute
# X_train = np.asarray(X_train).astype('float32')
# X_test = np.asarray(X_test).astype('float32')

history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    verbose=1)

Epoch 1/20


2021-12-10 14:23:59.465787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


262/262 [==============================] - ETA: 0s - loss: 0.5358 - accuracy: 0.6023

2021-12-10 14:24:03.908905: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


262/262 [==============================] - 5s 15ms/step - loss: 0.5358 - accuracy: 0.6023 - val_loss: 0.4790 - val_accuracy: 0.6151
Epoch 2/20
262/262 [==============================] - 3s 13ms/step - loss: 0.3533 - accuracy: 0.6236 - val_loss: 0.3688 - val_accuracy: 0.6084
Epoch 3/20
262/262 [==============================] - 3s 13ms/step - loss: 0.1569 - accuracy: 0.6420 - val_loss: 0.2480 - val_accuracy: 0.6261
Epoch 4/20
262/262 [==============================] - 3s 13ms/step - loss: -0.1778 - accuracy: 0.6501 - val_loss: 0.1083 - val_accuracy: 0.6447
Epoch 5/20
262/262 [==============================] - 3s 13ms/step - loss: -0.8081 - accuracy: 0.6666 - val_loss: -0.1054 - val_accuracy: 0.6480
Epoch 6/20
262/262 [==============================] - 3s 13ms/step - loss: -2.0426 - accuracy: 0.6750 - val_loss: -0.4193 - val_accuracy: 0.5903
Epoch 7/20
262/262 [==============================] - 3s 13ms/step - loss: -4.4595 - accuracy: 0.6749 - val_loss: -0.9020 - val_accuracy: 0.6289
Epo